In [1]:
import numpy as np
import pandas as pd
from Bio import SeqIO

from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
data_path = Path("../data/cafa5")
list(data_path.iterdir())

[PosixPath('../data/cafa5/test_split.parquet'),
 PosixPath('../data/cafa5/train_bp_top500_seqs.parquet'),
 PosixPath('../data/cafa5/top100_test_split.parquet'),
 PosixPath('../data/cafa5/train_bp_top100_targets.npy'),
 PosixPath('../data/cafa5/top100_train_split.parquet'),
 PosixPath('../data/cafa5/train_bp_top100_entryids.npy'),
 PosixPath('../data/cafa5/train_bp_top100_terms.npy'),
 PosixPath('../data/cafa5/train_bp_top100_seqs.parquet'),
 PosixPath('../data/cafa5/train_bp_top500_targets.npy'),
 PosixPath('../data/cafa5/train_bp_top500_terms.npy'),
 PosixPath('../data/cafa5/cafa-5-protein-function-prediction'),
 PosixPath('../data/cafa5/train_split.parquet'),
 PosixPath('../data/cafa5/train_bp_top500_entryids.npy')]

In [3]:
train_df = pd.read_csv(
    data_path / "cafa-5-protein-function-prediction/Train/train_terms.tsv", sep="\t"
)
print(train_df.shape)
train_df.head()

(5363863, 3)


,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO


In [4]:
train_df["aspect"].value_counts()

aspect
BPO    3497732
CCO    1196017
MFO     670114
Name: count, dtype: int64

In [5]:
# Biological Process only
bp = train_df.loc[train_df["aspect"] == "BPO"]
bp.shape

(3497732, 3)

In [6]:
print(f"Number of unique proteins: {len(bp['EntryID'].unique())}")
print(f"Unique GO terms for BP: {len(bp['term'].unique())}")

Number of unique proteins: 92210
Unique GO terms for BP: 21285


In [7]:
bp_term_counts = bp.groupby("term")["EntryID"].count().sort_values(ascending=False)
bp_term_counts

term
GO:0008150    92210
GO:0009987    61293
GO:0065007    41457
GO:0050789    39256
GO:0050794    33888
              ...  
GO:0071231        1
GO:1901233        1
GO:0048034        1
GO:0048033        1
GO:0099083        1
Name: EntryID, Length: 21285, dtype: int64

In [8]:
# take the most frequent terms
top = 100
top_terms = bp_term_counts[:top].index.tolist()
top_terms[:5]

['GO:0008150', 'GO:0009987', 'GO:0065007', 'GO:0050789', 'GO:0050794']

In [9]:
# select entries based on top_terms
bp_top = bp.loc[bp["term"].isin(top_terms)]
bp_top.shape

(1423484, 3)

In [10]:
# # of labels per protein
num_labels = bp_top.groupby("EntryID")["term"].count().sort_values(ascending=False)
num_labels

EntryID
P02340    92
P01137    91
Q01705    88
Q04887    88
P06802    87
          ..
Q5MK87     1
Q9VLC3     1
Q9CXF3     1
P24698     1
Q9WV42     1
Name: term, Length: 92210, dtype: int64

In [11]:
num_labels.mean()

15.43741459711528

In [12]:
(num_labels > 2).sum()

88652

In [13]:
# select proteins with at least a few terms
bp_top = bp_top.loc[bp_top["EntryID"].isin(num_labels[num_labels > 10].index.tolist())]
bp_top.shape

(1170659, 3)

In [14]:
print(f"Number of unique proteins after selection: {len(bp_top['EntryID'].unique())}")

Number of unique proteins after selection: 52593


In [15]:
# generate a set of terms for each protein
labels = bp_top.groupby("EntryID")["term"].apply(list)
labels

EntryID
A0A009IHW8    [GO:0008152, GO:0044237, GO:1901360, GO:000815...
A0A021WW32    [GO:0048869, GO:0048856, GO:0065007, GO:000727...
A0A023GPK8    [GO:0032502, GO:0050793, GO:0048856, GO:006500...
A0A023GRW4    [GO:0009605, GO:0048869, GO:0048856, GO:000727...
A0A023GU64    [GO:0048856, GO:0007275, GO:0009653, GO:000815...
                                    ...                        
X2JL73        [GO:0009605, GO:0050793, GO:0065008, GO:006500...
X2KK07        [GO:0009605, GO:0065007, GO:0008150, GO:004222...
X2KN52        [GO:0009605, GO:0065007, GO:0008150, GO:004222...
X4YX01        [GO:0048869, GO:0051641, GO:0065007, GO:004885...
X5M5N0        [GO:0009893, GO:0060255, GO:0065008, GO:000962...
Name: term, Length: 52593, dtype: object

In [16]:
# multi-label one-hot encoding targets
mlb = MultiLabelBinarizer(classes=top_terms)

targets = mlb.fit_transform(labels.tolist())
targets.shape

(52593, 100)

In [17]:
targets[0]

array([1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [18]:
# save to files
np.save(data_path / f"train_bp_top{top}_targets.npy", targets)
np.save(data_path / f"train_bp_top{top}_entryids.npy", labels.index.to_numpy())
np.save(data_path / f"train_bp_top{top}_terms.npy", np.array(top_terms))

In [19]:
# sequence data
entry_ids = []
seqs = []
bp_entry_ids = set(labels.index)

for record in SeqIO.parse(
    data_path / "cafa-5-protein-function-prediction/Train/train_sequences.fasta",
    "fasta",
):
    if record.id in bp_entry_ids:
        entry_ids.append(record.id)
        seqs.append(str(record.seq))

In [20]:
df = pd.DataFrame(seqs, index=entry_ids, columns=["Sequence"])
df.head()

,Sequence
P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...
O73864,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...
O95231,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...
A0A0B4J1F4,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...
P33681,MGHTRRQGTSPSKCPYLNFFQLLVLAGLSHFCSGVIHVTKEVKEVA...


In [21]:
# ensure the same order as targets
df = df.loc[labels.index.tolist()]
df.reset_index(names="Entry ID", inplace=True)
df["Index"] = df.index.values
df

,Entry ID,Sequence,Index
0,A0A009IHW8,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,0
1,A0A021WW32,MFYEHIILAKKGPLARIWLAAHWDKKITKAHVFETNIEKSVEGILQ...,1
2,A0A023GPK8,MSTIKLLIIGQLWLSIGLISGDDSLDTREGVDLVLKCRFTEHYDST...,2
3,A0A023GRW4,MMGSPGSQASAIATSVGIRSGRRGQAGGSLLLRLLAVTFVLAACHA...,3
4,A0A023GU64,MDYTRTHRAEAMHSCLSRMRFRCFFLMLSFFCEAAAFNLDLEKPTV...,4
...,...,...,...
52588,X2JL73,MASQHSSWRFGKRSKLQLRIKVSQDPEDFYRLDPQRSAAENAFEIY...,52588
52589,X2KK07,MAQWEKLRQLDSVYLTQVDELYDGDAFPMDVRHYLAHWIEGQDWDR...,52589
52590,X2KN52,MAQWEKLRQLDSVYLTQVDELYDGDAFPMDVRHYLAHWIEGQDWDR...,52590
52591,X4YX01,MEQDAAAGPAKKSGTTLSVDRAAFLLLRVRKVFKKKRQQRKERQAQ...,52591


In [22]:
from sklearn.model_selection import train_test_split

In [23]:
# create a holdout test set
test_size = 0.2
train_df, test_df = train_test_split(df, test_size=test_size, random_state=0)

In [24]:
df.to_parquet(data_path / f"train_bp_top{top}_seqs.parquet")
train_df.to_parquet(data_path / f"top{top}_train_split.parquet")
test_df.to_parquet(data_path / f"top{top}_test_split.parquet")